In [7]:
# ============================
# 📦 Install Required Packages
# ============================
!pip install osmnx folium networkx matplotlib numpy pandas geopandas shapely tqdm gtfs_kit

# ============================
# 📚 Imports
# ============================
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm
import folium
import gtfs_kit as gk

# ============================
# ⚙️ Colab / Notebook Settings
# ============================
ox.settings.use_cache = True
ox.settings.log_console = False
plt.style.use("seaborn-v0_8")

print("✔️ All libraries successfully installed and imported.")


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.5/101.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.6/507.6 kB 13.3 MB/s eta 0:00:00
  Created wheel for json2html: filename=json2html-1.3.0-py3-none-any.whl size=7591 sha256=3bc93df0955c41cfda3389380ee943ae748c3a47735350a11544596dc2b5f540
  Stored in directory: /root/.cache/pip/wheels/91/96/d0/3cfff4cc9a9e307cfc2b8fe4846c56d714b78bbe5f6da211a5
Successfully built json2html
✔️ All libraries successfully installed and imported.


In [8]:
from google.colab import drive
drive.mount('/content/drive')

GTFS_PARIS_DIR = "/content/drive/MyDrive/SNN_Project/Paris_gtfs/"
GTFS_PRAGUE_DIR = "/content/drive/MyDrive/SNN_Project/Prague_gtfs/PID_GTFS/"


def load_all_gtfs_tables(feed_dir):
    """
    Loads all .txt files in a GTFS directory into a dict of DataFrames.
    Key = file name without '.txt', e.g. 'stops', 'routes', 'vehicle_categories', ...
    """
    gtfs = {}
    for fname in os.listdir(feed_dir):
        if fname.lower().endswith(".txt"):
            path = os.path.join(feed_dir, fname)
            key = fname[:-4]  # remove '.txt'
            df = pd.read_csv(path)          # default: comma-separated

            gtfs[key] = df
            print(f"Loaded {key:25s} → shape = {df.shape}")
    return gtfs


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
gtfs_paris  = load_all_gtfs_tables(GTFS_PARIS_DIR)
gtfs_prague = load_all_gtfs_tables(GTFS_PRAGUE_DIR)

Loaded object_codes_extension    → shape = (66398, 4)
Loaded transfers                 → shape = (6894, 4)
Loaded stops                     → shape = (5437, 14)
Loaded stop_times                → shape = (696956, 14)
Loaded stop_extensions           → shape = (16574, 3)
Loaded pathways                  → shape = (5680, 12)
Loaded calendar                  → shape = (484, 10)
Loaded agency                    → shape = (3, 8)
Loaded calendar_dates            → shape = (1106, 3)
Loaded routes                    → shape = (42, 10)
Loaded trips                     → shape = (49686, 10)
Loaded levels                    → shape = (171, 3)
Loaded fare_attributes           → shape = (17, 7)
Loaded vehicle_couplings         → shape = (5, 3)
Loaded vehicle_allocations       → shape = (3, 2)
Loaded vehicle_categories        → shape = (3, 1)
Loaded vehicle_boardings         → shape = (230, 3)
Loaded pathways                  → shape = (1497, 9)


/tmp/ipython-input-2841693398.py:18: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)          # default: comma-separated


Loaded trips                     → shape = (79810, 12)
Loaded stops                     → shape = (18617, 14)
Loaded routes                    → shape = (841, 11)
Loaded agency                    → shape = (1, 6)
Loaded calendar_dates            → shape = (15311, 3)
Loaded calendar                  → shape = (654, 10)


/tmp/ipython-input-2841693398.py:18: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)          # default: comma-separated


Loaded stop_times                → shape = (1587704, 11)
Loaded fare_rules                → shape = (38, 3)
Loaded feed_info                 → shape = (1, 6)
Loaded route_stops               → shape = (35406, 4)
Loaded transfers                 → shape = (12404, 7)
Loaded shapes                    → shape = (3556090, 5)
Loaded route_sub_agencies        → shape = (890, 4)


In [10]:
gtfs_paris.keys()       # list of all available tables for Paris

dict_keys(['object_codes_extension', 'transfers', 'stops', 'stop_times', 'stop_extensions', 'pathways', 'calendar', 'agency', 'calendar_dates', 'routes', 'trips'])

In [11]:
gtfs_prague.keys()     # list of all available tables for Prague

dict_keys(['levels', 'fare_attributes', 'vehicle_couplings', 'vehicle_allocations', 'vehicle_categories', 'vehicle_boardings', 'pathways', 'trips', 'stops', 'routes', 'agency', 'calendar_dates', 'calendar', 'stop_times', 'fare_rules', 'feed_info', 'route_stops', 'transfers', 'shapes', 'route_sub_agencies'])